In [6]:
import numpy as np
import peewee as pw
import pandas as pd

from tqdm import tqdm
from structure import Const, FuncData, Solve, database_connection
from solve import default_values, const_keys, path

In [7]:
def df_from_comsol(name):
    with open(path + r'\Batch\io\Data.csv', 'r') as file:
        lines = file.readlines()
    lines = lines[4:]

    lines[:] = (line.replace('\n', '') for line in lines[:])
    lines[0] = lines[0].replace('% ', '')

    data = [line.split(',') for line in lines]
    return pd.DataFrame(data=data[1:], columns=data[0]).astype(float)


In [8]:
df = df_from_comsol('test')
df.head()

,Ke,KH,Kr,Kdisp,KqH,Ks,Kd,Kc,Kp,KrD,...,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
0,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
1,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,2.000000e-12,0.0,0.0,0.0,0.0,0.0
2,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,4.000000e-12,0.0,0.0,0.0,0.0,0.0
3,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,8.000000e-12,0.0,0.0,0.0,0.0,0.0
4,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,2000.0,3.0,0.05,1.0,0.001,1.000000e+09,...,1.0,1.0,0.0,0.0,1.600000e-11,0.0,0.0,0.0,0.0,0.0


In [9]:
def check(string):
    if string is None:
        while string != 'q':
            string = input(f'Set {string=}, to quit - q:')
            string = string.strip()
    return string


def write(name, desc, df):
    grouped_frames = []
    nonconst_keys = [i for i in df.columns if i not in const_keys]
    name = check(name)
    desc = check(desc)

    with database_connection:
        i = 0
        groups = tqdm(iterable=df.groupby(by=const_keys))
        for const_values, frame in groups:
            # Solve
            solve = Solve.create(name=name + f'_#{i}', desc=desc)

            # Consts
            _zipped = zip(const_keys, const_values)
            _consts_q = [{
                'solve': solve,
                'name': key,
                'value': value,
            } for key,value in _zipped] # yapf:disable
            Const.insert_many(_consts_q).execute()

            # Functions
            func_frame = frame[nonconst_keys]
            func_frame = func_frame.reset_index(drop=True)
            FuncData.create(
                solve=solve,
                data=func_frame.to_json(index=True),
            )

            i += 1
            grouped_frames.append(func_frame)
    return grouped_frames

In [10]:
a=write('test','test decs  ',df)

100%|██████████| 12/12 [00:01<00:00, 11.81it/s]


In [38]:
with database_connection:
    q= FuncData.select(FuncData).join(Solve).where(Solve.name=='test_#11').get()

        
pd.read_json(q.data)

,Time,Q,DH,QHD,QHH,prod,Qm,DHp,QH,D,QD
0,0.0002,0.328433,0.000455,0.342859,0.328045,0.328227,0.00013,0.00013,0.000465,0.000034,0.000068


In [8]:
def distribute(df: pd.DataFrame):
    condition = lambda i: ('K' in i) or ('light' in i)
    consts = [i for i in df.columns if condition(i)]
    nonconsts = [i for i in df.columns if not condition(i)]
    combs = df.drop_duplicates(subset=consts)[consts]
    combs = combs.reset_index(drop=True)
    notes = []
    for num, comb in combs.iterrows():

        df_filtered = df
        for key, value in comb.items():
            df_filtered = df_filtered[df_filtered[key] == value]

        df_filtered = df_filtered.reset_index(drop=True)
        const_vals = dict(zip(comb.keys(), comb.values))
        notes.append((const_vals, df_filtered[nonconsts]))
    return notes

In [6]:
# def save(name=None, desc=None):
#     if name is None:
#         while name != 'q':
#             name = input(f'Set {name=}, to quit - q:')

#     if desc is None:
#         while desc != 'q':
#             desc = input(f'Set {desc=}, to quit - q:')

#     with database_connection:
#         q = Solves(name = name,description=desc)
#         q.save()


In [ ]:
with database_connection:
    solve = Solve.create(name='test', description=None)

    for key, value in a[1][0].items():
        coef = Coefs.create(solve=solve, key=key, value=value)


In [24]:
# Series
[{'solve': '1', 'key': key, 'value': value} for key, value in b.items()]


[{'solve': '1', 'key': 'Ke', 'value': 1000000000.0},
 {'solve': '1', 'key': 'KH', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kr', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kdisp', 'value': 1000000000.0},
 {'solve': '1', 'key': 'KqH', 'value': 2000.0},
 {'solve': '1', 'key': 'Ks', 'value': 3.0},
 {'solve': '1', 'key': 'Kd', 'value': 0.05},
 {'solve': '1', 'key': 'Kc', 'value': 1.0},
 {'solve': '1', 'key': 'Kp', 'value': 0.001},
 {'solve': '1', 'key': 'KrD', 'value': 1000000000.0},
 {'solve': '1', 'key': 'Kph', 'value': 1e-05},
 {'solve': '1', 'key': 'light', 'value': 0.0}]